In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

import warnings
warnings.filterwarnings('ignore')

# Utils
file_path = os.getcwd()  # os.path.dirname(os.path.relpath(__file__))
utils_path = os.path.abspath(os.path.join(file_path, '../src/data'))
sys.path.append(utils_path)

# import utils_models
import utils_data as utils


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
DATADIR = '/Users/apartin/work/jdacs/Benchmarks/Data/Pilot1'

RSP_FILENAME = 'combined_single_response_agg'  # reposne data filename
# RSP_FILENAME = 'ChemPartner_single_response_agg'  # reposne data filename

#DSC_FILENAME = 'Combined_PubChem_dragon7_descriptors.tsv'  # drug descriptors data filename
DSC_FILENAME = 'pan_drugs_dragon7_descriptors.tsv'

DRUG_META_FILENAME = 'drug_info'


In [19]:
# ========================================================================
#       Args TODO: add to argparse
# ========================================================================
# sources = ['ccle', 'gcsi', 'gdsc', 'ctrp']
sources = ['ccle', 'gcsi', 'gdsc', 'ctrp', 'nci60']
drug_features = ['dsc']  # ['dsc', 'fng']
cell_features = ['rna']  # ['rna', 'cnv']
dropna_thres = 0.4

verbose = True
na_values = ['na', '-', '']
tidy_data_format = 'parquet'

# Analysis of fibro samples are implemented in ccle_fibroblast.py and ccle_preproc.R
drop_fibro = True
fibro_names = ['CCLE.HS229T', 'CCLE.HS739T', 'CCLE.HS840T', 'CCLE.HS895T', 'CCLE.RKN',
               'CTRP.Hs-895-T', 'CTRP.RKN', 'GDSC.RKN', 'gCSI.RKN']

# Prefix to add to feature names based on feature types
fea_prfx_dict = {'rna': 'cell_rna.',
                 'cnv': 'cell_cnv.',
                 'dsc': 'drug_dsc.',
                 'fng': 'drug_fng.'}

prfx_dtypes = {'rna': np.float32,
               'cnv': np.int8,
               'dsc': np.float32,
               'fng': np.int8}


In [20]:
# ========================================================================
#       Load response data
# ========================================================================
print(f'Loading combined response ... {RSP_FILENAME}')
rsp_cols = ['AUC', 'AUC1', 'EC50', 'EC50se',
            'R2fit', 'Einf', 'IC50',
            'HS', 'AAC1', 'DSS1']
rsp = pd.read_table(os.path.join(DATADIR, RSP_FILENAME), sep='\t',
                    na_values=na_values,
                    dtype={'SOURCE': str, 'CELL': str, 'DRUG': str,
                           'AUC': np.float32, 'IC50': np.float32, 'EC50': np.float32,
                           'EC50se': np.float32, 'R2fit': np.float32, 'Einf': np.float32,
                           'HS': np.float32, 'AAC1': np.float32, 'AUC1': np.float32, 'DSS1': np.float32},
                    warn_bad_lines=True)
rsp['SOURCE'] = rsp['SOURCE'].apply(lambda x: x.lower())
print(f'rsp.shape {rsp.shape}')

# Replace -Inf and Inf with nan
rsp.replace([np.inf, -np.inf], value=np.nan, inplace=True)

print(rsp.groupby('SOURCE').agg({'CELL': 'nunique', 'DRUG': 'nunique'}).reset_index())


Loading combined response ... combined_single_response_agg
rsp.shape (4484081, 14)
  SOURCE  CELL   DRUG
0   ccle   504     24
1   ctrp   887    544
2   gcsi   409     16
3   gdsc  1075    249
4  nci60    59  52671
5    scl    65    445
6   sclc    70    526


In [21]:
# ========================================================================
#   Load rna (combined_dataset)
# ========================================================================
print('Loading rna-seq ... ')
lincs = utils.CombinedRNASeqLINCS(datadir=DATADIR, dataset='raw', sources=sources, na_values=na_values, verbose=False)
rna, cmeta = lincs._df_rna, lincs._meta
rna.rename(columns={'Sample': 'CELL'}, inplace=True)
cmeta.rename(columns={'Sample': 'CELL', 'source': 'SOURCE'}, inplace=True)
rna = rna.rename(columns={c: fea_prfx_dict['rna']+c for c in rna.columns[1:] if fea_prfx_dict['rna'] not in c}) # add fea prefix
print(f'rna.shape {rna.shape}')

# Impute missing values
rna = utils.impute_values(data=rna, fea_prfx_dict=fea_prfx_dict)

print(cmeta.groupby('SOURCE').agg({'CELL': 'nunique', 'ctype': 'nunique', 'csite': 'nunique'}).reset_index())


Loading rna-seq ... 
rna.shape (2917, 943)
  SOURCE  CELL  ctype  csite
0   ccle  1018     65     27
1   ctrp   812     55     24
2   gcsi   357     40     22
3   gdsc   670     44     24
4  nci60    60     20      9


In [22]:
# ========================================================================
#   Load drug descriptors
# ========================================================================
print(f'Loading drug descriptors ... {DSC_FILENAME}')
path = os.path.join(DATADIR, DSC_FILENAME)
cols = pd.read_table(path, engine='c', nrows=0)
dtype_dict = {c: prfx_dtypes['dsc'] for c in cols.columns[1:]}
dsc = pd.read_table(path, dtype=dtype_dict, na_values=na_values, warn_bad_lines=True)
#dsc.rename(columns={'NAME': 'PUBCHEM'}, inplace=True) # used in the old code
dsc.rename(columns={'NAME': 'DRUG'}, inplace=True)
dsc = dsc.rename(columns={c: fea_prfx_dict['dsc']+c for c in dsc.columns[1:] if fea_prfx_dict['dsc'] not in c}) # add fea prefix
print(f'dsc.shape {dsc.shape}')


# ------------------
# Filter descriptors
# ------------------
# dsc.nunique(dropna=True).value_counts()
# dsc.nunique(dropna=True).sort_values()

print('Drop descriptors with *lots* of NA values ...')
#utils.plot_dsc_na_dist(dsc=dsc, savepath=os.path.join(OUTDIR, 'dsc_hist_ratio_of_na.png'))
dsc = utils.dropna(df=dsc, axis=1, th=dropna_thres)
print(f'dsc.shape {dsc.shape}')
# dsc.isna().sum().sort_values(ascending=False)

# There are descriptors for which there is a single unique value excluding NA (drop those)
print('Drop descriptors that have a single unique value (excluding NAs) ...')
col_idx = dsc.nunique(dropna=True).values==1
dsc = dsc.iloc[:, ~col_idx]
print(f'dsc.shape {dsc.shape}')

# There are still lots of descriptors which have only a few unique values
# We can categorize those values. E.g.: 564 descriptors have only 2 unique vals,
# and 154 descriptors have only 3 unique vals, etc.
# TODO: use utility code from p1h_alex/utils/data_preproc.py that transform those
# features into categorical and also applies an appropriate imputation.
# dsc.nunique(dropna=True).value_counts()[:10]
# dsc.nunique(dropna=True).value_counts().sort_index()[:10]

# Impute missing values
dsc = utils.impute_values(data=dsc, fea_prfx_dict=fea_prfx_dict)

# Drop low var cols
# tmp, idx = utils_all.drop_low_var_cols(df=dsc, skipna=False)

if verbose:
    print('dsc memory usage: {:.3f} GB'.format(sys.getsizeof(dsc)/1e9))


Loading drug descriptors ... pan_drugs_dragon7_descriptors.tsv
dsc.shape (1801, 5271)
Drop descriptors with *lots* of NA values ...
dsc.shape (1801, 3838)
Drop descriptors that have a single unique value (excluding NAs) ...
dsc.shape (1801, 2821)
dsc memory usage: 0.020 GB


In [23]:
# ========================================================================
#   Load drug meta
# ========================================================================
""" We don't need drug meta with the new descriptors file. """
# DRUG_META_FILENAME = 'drug_info'
# print(f'Loading drug metadata ... {DRUG_META_FILENAME}')
# dmeta = pd.read_table(os.path.join(DATADIR, DRUG_META_FILENAME), dtype=object)
# dmeta['PUBCHEM'] = 'PubChem.CID.' + dmeta['PUBCHEM']
# dmeta.insert(loc=0, column='SOURCE', value=dmeta['ID'].map(lambda x: x.split('.')[0].lower()))
# dmeta.rename(columns={'ID': 'DRUG'}, inplace=True)
# print(f'dmeta.shape {dmeta.shape}')


" We don't need drug meta with the new descriptors file. "

In [24]:
# ========================================================================
#   Drop fibroblast
# ========================================================================
if drop_fibro:
    print('Drop fibroblast samples ...')
    rna = rna[rna['CELL'].map(lambda x: False if x in fibro_names else True)]
    cmeta = cmeta[cmeta['CELL'].map(lambda x: False if x in fibro_names else True)]
    rsp = rsp[rsp['CELL'].map(lambda x: False if x in fibro_names else True)]
    print(f'rsp.shape   {rsp.shape}')
    print(f'rna.shape   {rna.shape}')
    print(f'cmeta.shape {cmeta.shape}')

    tidy_data_name = 'tidy_data_no_fibro'
else:
    tidy_data_name = 'tidy_data'

Drop fibroblast samples ...
rsp.shape   (4482796, 14)
rna.shape   (2908, 943)
cmeta.shape (2908, 20)


### Dump shapes

In [25]:
rsp.iloc[:2]

,SOURCE,CELL,DRUG,STUDY,AUC,IC50,EC50,EC50se,R2fit,Einf,HS,AAC1,AUC1,DSS1
0,ccle,CCLE.1321N1,CCLE.1,fake_exp,0.8330,4.883,5.183,0.5746,0.9864,0.2017,0.7463,0.1017,0.8983,0.0551
1,ccle,CCLE.1321N1,CCLE.10,fake_exp,0.7909,5.217,5.217,2.2840,0.6526,0.0000,0.8993,0.1136,0.8864,0.0728


In [26]:
rna.iloc[:2, :5]

,CELL,cell_rna.AARS,cell_rna.ABCB6,cell_rna.ABCC5,cell_rna.ABCF1
0,CCLE.22RV1,8.31,7.17,4.12,5.64
1,CCLE.2313287,8.94,6.30,3.83,6.60


In [27]:
dsc.iloc[:2, :5]

,DRUG,drug_dsc.MW,drug_dsc.AMW,drug_dsc.Sv,drug_dsc.Se
0,CCLE.10,475.399994,8.804,34.717999,54.522999
1,CCLE.11,349.470001,7.132,30.761999,48.796001


In [28]:
# dmeta.iloc[:2]

In [29]:
# ========================================================================
#   Merge the data
# ========================================================================
"""
Data tables: rsp, rna, dsc, cmeta, dmeta
(rsp, rna): on 'CELL'
(rsp, dsc): on pubchem through fields in dmeta
"""
print('==========================')
print('... Start merging data ...')
print('==========================')

# Update rsp with additional drug field 'PUBCHEM' (this will be used to merge with descriptors)
""" No need dmeta with the new descriptors file. """
# print('\nMerge response (rsp) with drug metadata (dmeta) on DRUG in order to add PUBCHEM (required for descriptors) ...')
# print(f'rsp.shape   {rsp.shape}')
# print(f'dmeta.shape {dmeta.shape}')
# rsp = pd.merge(rsp, dmeta[['DRUG', 'PUBCHEM']], on='DRUG', how='left')
# print(f'rsp.shape   {rsp.shape}')
# print('NA values after merging rsp and dmeta: \n{}'.format(rsp[['DRUG', 'PUBCHEM']].isna().sum()))
# print('')
# print(rsp.groupby('SOURCE').agg({'DRUG': 'nunique', 'PUBCHEM': 'nunique'}).reset_index())


... Start merging data ...


' No need dmeta with the new descriptors file. '

In [30]:
# --------------------
# Merge rsp with cmeta
# --------------------
print('\nMerge response (rsp) and cell metadata (cmeta) ...')
print(f'rsp.shape   {rsp.shape}')
print(f'cmeta.shape {cmeta.shape}')
rsp1 = pd.merge(rsp, cmeta[['CELL', 'core_str', 'csite', 'ctype', 'simplified_csite', 'simplified_ctype']], on='CELL', how='left')
print(f'rsp1.shape  {rsp1.shape}')
print('')
# print(rsp1.groupby('SOURCE').agg({'CELL': 'nunique', 'DRUG': 'nunique',
#                                         'PUBCHEM': 'nunique'}).reset_index())
print(rsp1.groupby('SOURCE').agg({'CELL': 'nunique'}).reset_index())
del rsp



Merge response (rsp) and cell metadata (cmeta) ...
rsp.shape   (4482796, 14)
cmeta.shape (2908, 20)
rsp1.shape  (4482796, 19)

  SOURCE  CELL
0   ccle   499
1   ctrp   885
2   gcsi   408
3   gdsc  1074
4  nci60    59
5    scl    65
6   sclc    70


In [31]:
# --------------
# Merge with rna
# --------------
print('\nMerge with expression (rna) ...')
print(f'rsp1.shape {rsp1.shape}')
print(f'rna.shape  {rna.shape}')
rsp2 = pd.merge(rsp1, rna, on='CELL', how='inner')
print(f'rsp2.shape {rsp2.shape}')
print('')
print(rsp2.groupby('SOURCE').agg({'CELL': 'nunique'}).reset_index())
del rsp1



Merge with expression (rna) ...
rsp1.shape (4482796, 19)
rna.shape  (2908, 943)
rsp2.shape (4299936, 961)

  SOURCE  CELL
0   ccle   469
1   ctrp   810
2   gcsi   356
3   gdsc   669
4  nci60    59


In [32]:
# --------------
# Merge with dsc
# --------------
print('\nMerge with descriptors (dsc) ...')
print(f'rsp2.shape {rsp2.shape}')
print(f'dsc.shape  {dsc.shape}')
#data = pd.merge(rsp2, dsc, on='PUBCHEM', how='inner')
data = pd.merge(rsp2, dsc, on='DRUG', how='inner')
print(f'data.shape {data.shape}')
print('')
print(data.groupby('SOURCE').agg({'CELL': 'nunique', 'DRUG': 'nunique'}).reset_index())
del rsp2



Merge with descriptors (dsc) ...
rsp2.shape (4299936, 961)
dsc.shape  (1801, 2821)
data.shape (704153, 3781)

  SOURCE  CELL  DRUG
0   ccle   469    24
1   ctrp   810   495
2   gcsi   356    16
3   gdsc   669   239
4  nci60    59  1006


In [34]:
data.iloc[:3, :4]

,SOURCE,CELL,DRUG,STUDY
0,ccle,CCLE.22RV1,CCLE.1,fake_exp
1,ccle,CCLE.42MGBA,CCLE.1,fake_exp
2,ccle,CCLE.5637,CCLE.1,fake_exp


In [40]:
df = data.copy()

In [41]:
print(df.shape)
target_name = 'AUC'
df = df[~df[target_name].isna()]
print(df.shape)

(704153, 3781)
(704152, 3781)
